In [189]:
import pandas as pd
import altair as alt
import altair_latimes as lat

In [190]:
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

### Import

District data downloaded from portal

In [191]:
df = pd.read_csv("../data/raw/water-use-by-district-timeseries.csv", parse_dates=["reporting_month"])

Regional data scraped from dashboard 

In [192]:
regional_usage = pd.read_csv("../data/processed/regional-residential-usage.csv", parse_dates=["reporting_month"])

In [193]:
regional_usage

,hydrologic_region,reporting_month,agencies_reporting,pop_weighted_rgpcd
0,Central Coast,2014-06-15,30,100
1,Colorado River,2014-06-15,13,222
2,North Coast,2014-06-15,13,88
3,North Lahontan,2014-06-15,5,162
4,Sacramento River,2014-06-15,40,187
...,...,...,...,...
935,San Francisco Bay,2022-03-15,43,59
936,San Joaquin River,2022-03-15,22,93
937,South Coast,2022-03-15,164,76
938,South Lahontan,2022-03-15,16,90


---
### Clean and analyze

In [194]:
df.columns

Index(['supplier_name', 'public_water_system_id', 'reporting_month', 'county',
       'hydrologic_region', 'climate_zone', 'total_population_served',
       'reference_2014_population', 'county_under_drought_declaration',
       'water_shortage_contingency_stage_invoked',
       'water_shortage_level_indicator',
       'dwr_state_standard_level_corresponding_to_stage',
       'water_production_units',
       'reported_preliminary_total_potable_water_production',
       'reported_final_total_potable_water_production',
       'preliminary_percent_residential_use', 'final_percent_residential_use',
       'reported_preliminary_commercial_agricultural_water',
       'reported_final_commercial_agricultural_water',
       'reported_preliminary_commercial_industrial_and_institutional_water',
       'reported_final_commercial_industrial_and_institutional_water',
       'reported_recycled_water', 'reported_non_revenue_water',
       'calculated_total_potable_water_production_gallons_ag_excluded'

In [281]:
test = df[['supplier_name', 'reporting_month',
       'hydrologic_region', 'total_population_served',
        'water_production_units',
        'reported_preliminary_total_potable_water_production',
       'reported_final_total_potable_water_production',
        'preliminary_percent_residential_use', 'final_percent_residential_use',
       'reported_preliminary_commercial_agricultural_water',
       'reported_final_commercial_agricultural_water',
       'reported_preliminary_commercial_industrial_and_institutional_water',
       'reported_final_commercial_industrial_and_institutional_water',
       'calculated_total_potable_water_production_gallons_ag_excluded',
       'calculated_r_gpcd']].copy()

In [282]:
test.water_production_units.unique()

array(['MG', 'G', 'AF', 'CCF'], dtype=object)

In [283]:
to = test[test["supplier_name"] == "Thousand Oaks  City of"]

In [284]:
to["reported_final_total_potable_water_production_gal"] = to["reported_final_total_potable_water_production"] * 325851

/var/folders/7p/v902smb50m50mm_swb_lj1rh0000gn/T/ipykernel_25087/1215496093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to["reported_final_total_potable_water_production_gal"] = to["reported_final_total_potable_water_production"] * 325851


In [285]:
to["residential_gal"] = (to["final_percent_residential_use"]/100)*to["reported_final_total_potable_water_production_gal"]

/var/folders/7p/v902smb50m50mm_swb_lj1rh0000gn/T/ipykernel_25087/141913974.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to["residential_gal"] = (to["final_percent_residential_use"]/100)*to["reported_final_total_potable_water_production_gal"]


In [286]:
to["commercial_gal"] = to["reported_final_commercial_industrial_and_institutional_water"] * 325851

/var/folders/7p/v902smb50m50mm_swb_lj1rh0000gn/T/ipykernel_25087/3415809710.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to["commercial_gal"] = to["reported_final_commercial_industrial_and_institutional_water"] * 325851


In [287]:
to

,supplier_name,reporting_month,hydrologic_region,total_population_served,water_production_units,reported_preliminary_total_potable_water_production,reported_final_total_potable_water_production,preliminary_percent_residential_use,final_percent_residential_use,reported_preliminary_commercial_agricultural_water,reported_final_commercial_agricultural_water,reported_preliminary_commercial_industrial_and_institutional_water,reported_final_commercial_industrial_and_institutional_water,calculated_total_potable_water_production_gallons_ag_excluded,calculated_r_gpcd,reported_final_total_potable_water_production_gal,residential_gal,commercial_gal
22807,Thousand Oaks City of,2022-03-15,South Coast,53157.0,AF,NaN,785.4,NaN,79.95,NaN,0.0,NaN,117.20,255923375.4,124.1040,255923375.4,2.046107e+08,38189737.20
22808,Thousand Oaks City of,2022-02-15,South Coast,53157.0,AF,NaN,704.2,NaN,79.94,NaN,0.0,NaN,117.17,229464274.2,123.2080,229464274.2,1.834337e+08,38179961.67
22809,Thousand Oaks City of,2022-01-15,South Coast,53157.0,AF,NaN,571.1,NaN,83.26,NaN,0.0,NaN,103.10,186093506.1,94.0091,186093506.1,1.549415e+08,33595238.10
22810,Thousand Oaks City of,2021-12-15,South Coast,53157.0,AF,NaN,560.3,NaN,78.06,NaN,0.0,NaN,162.83,182574315.3,86.4399,182574315.3,1.425175e+08,53058318.33
22811,Thousand Oaks City of,2021-11-15,South Coast,53157.0,AF,NaN,823.0,NaN,75.36,NaN,0.0,NaN,220.99,268175373.0,126.7300,268175373.0,2.020970e+08,72009812.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22896,Thousand Oaks City of,2014-10-15,South Coast,51609.0,AF,NaN,1078.0,NaN,76.00,NaN,NaN,NaN,NaN,351267378.0,166.8650,351267378.0,2.669632e+08,NaN
22897,Thousand Oaks City of,2014-09-15,South Coast,51609.0,AF,NaN,1129.9,NaN,67.00,NaN,NaN,NaN,NaN,368179044.9,159.3410,368179044.9,2.466800e+08,NaN
22898,Thousand Oaks City of,2014-08-15,South Coast,51609.0,AF,NaN,1199.0,NaN,74.00,NaN,NaN,NaN,NaN,390695349.0,180.7110,390695349.0,2.891146e+08,NaN
22899,Thousand Oaks City of,2014-07-15,South Coast,51609.0,AF,NaN,1236.0,NaN,60.00,NaN,NaN,NaN,NaN,402751836.0,151.0440,402751836.0,2.416511e+08,NaN


In [288]:
days_per_month = {
    "1": 31,
    "2": 28,
    "3": 31,
    "4": 30,
    "5": 31,
    "6": 30,
    "7": 31,
    "8": 31,
    "9": 30,
    "10": 31,
    "11": 30,
    "12": 31
}

In [289]:
to["month"] = to["reporting_month"].dt.month.astype(str)

/var/folders/7p/v902smb50m50mm_swb_lj1rh0000gn/T/ipykernel_25087/291580525.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to["month"] = to["reporting_month"].dt.month.astype(str)


In [290]:
to["dpm"] = to["month"].map(days_per_month)

/var/folders/7p/v902smb50m50mm_swb_lj1rh0000gn/T/ipykernel_25087/162505744.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to["dpm"] = to["month"].map(days_per_month)


In [291]:
to["rgpcd"] = to["residential_gal"] / to["total_population_served"]/to["dpm"]

/var/folders/7p/v902smb50m50mm_swb_lj1rh0000gn/T/ipykernel_25087/2304132958.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to["rgpcd"] = to["residential_gal"] / to["total_population_served"]/to["dpm"]


In [295]:
to[["supplier_name", "calculated_r_gpcd", "rgpcd"]].head(50)

,supplier_name,calculated_r_gpcd,rgpcd
22807,Thousand Oaks City of,124.1040,124.167022
22808,Thousand Oaks City of,123.2080,123.242565
22809,Thousand Oaks City of,94.0091,94.025460
22810,Thousand Oaks City of,86.4399,86.486052
22811,Thousand Oaks City of,126.7300,126.729600
22812,Thousand Oaks City of,130.5250,130.524932
22813,Thousand Oaks City of,146.5690,146.599019
22814,Thousand Oaks City of,159.9520,159.952087
22815,Thousand Oaks City of,153.9550,153.926584
22816,Thousand Oaks City of,151.1130,151.141860


In [279]:
to[["supplier_name","residential_gal","commercial_gal","calculated_total_potable_water_production_gallons_ag_excluded"]]

,supplier_name,residential_gal,commercial_gal,calculated_total_potable_water_production_gallons_ag_excluded
22891,Thousand Oaks City of,1.878365e+08,NaN,264558426.9
22892,Thousand Oaks City of,1.474867e+08,NaN,210695256.6
22893,Thousand Oaks City of,1.651674e+08,NaN,229399104.0
22894,Thousand Oaks City of,1.270428e+08,NaN,167161563.0
22895,Thousand Oaks City of,1.973028e+08,NaN,281861115.0
22896,Thousand Oaks City of,2.669632e+08,NaN,351267378.0
22897,Thousand Oaks City of,2.466800e+08,NaN,368179044.9
22898,Thousand Oaks City of,2.891146e+08,NaN,390695349.0
22899,Thousand Oaks City of,2.416511e+08,NaN,402751836.0
22900,Thousand Oaks City of,2.344172e+08,NaN,390695349.0


In [296]:
test.groupby(["reporting_month", "supplier_name"])[["total_population_served","calculated_r_gpcd"]]

,supplier_name,reporting_month,hydrologic_region,total_population_served,water_production_units,reported_preliminary_total_potable_water_production,reported_final_total_potable_water_production,preliminary_percent_residential_use,final_percent_residential_use,reported_preliminary_commercial_agricultural_water,reported_final_commercial_agricultural_water,reported_preliminary_commercial_industrial_and_institutional_water,reported_final_commercial_industrial_and_institutional_water,calculated_total_potable_water_production_gallons_ag_excluded,calculated_r_gpcd
0,East Bay Municipal Utilities District,2022-03-15,San Francisco Bay,1430000.0,MG,4481.000,NaN,58.00,NaN,NaN,NaN,953.0,NaN,4.481000e+09,58.6280
1,East Bay Municipal Utilities District,2022-02-15,San Francisco Bay,1430000.0,MG,3898.000,NaN,54.00,NaN,NaN,NaN,888.0,NaN,3.898000e+09,52.5704
2,East Bay Municipal Utilities District,2022-01-15,San Francisco Bay,1420000.0,MG,3807.000,NaN,58.00,NaN,NaN,NaN,965.0,NaN,3.807000e+09,50.1604
3,East Bay Municipal Utilities District,2021-12-15,San Francisco Bay,1420000.0,MG,3754.000,NaN,60.00,NaN,NaN,NaN,821.0,NaN,3.754000e+09,51.1677
4,East Bay Municipal Utilities District,2021-11-15,San Francisco Bay,1420000.0,MG,3773.000,NaN,61.00,NaN,NaN,NaN,779.0,NaN,3.773000e+09,54.0265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37103,Thermalito Water and Sewer District,2021-05-15,Sacramento River,10312.0,CCF,120611.000,120611.000,81.05,81.05,0.0,0.0,16477.0,16477.0,9.022330e+07,228.7530
37104,Thermalito Water and Sewer District,2021-04-15,Sacramento River,10312.0,CCF,77407.487,77407.487,79.17,79.17,0.0,0.0,14195.0,14195.0,5.790483e+07,148.1860
37105,Thermalito Water and Sewer District,2021-03-15,Sacramento River,10312.0,CCF,45997.326,45997.326,83.21,83.21,0.0,0.0,7665.0,7665.0,3.440839e+07,89.5637
37106,Thermalito Water and Sewer District,2021-02-15,Sacramento River,10312.0,CCF,39363.636,39363.636,85.29,85.29,0.0,0.0,5341.0,5341.0,2.944605e+07,86.9818


In [195]:
keeps = ['supplier_name', 'reporting_month', 'county',
       'hydrologic_region', 'total_population_served',
       #'county_under_drought_declaration',
       # 'water_shortage_contingency_stage_invoked',
       #'water_shortage_level_indicator',
         'dwr_state_standard_level_corresponding_to_stage',
        'calculated_r_gpcd']

In [196]:
trim_df = df[keeps]

In [197]:
merge_df = pd.merge(
    trim_df, 
    regional_usage[["hydrologic_region","reporting_month","pop_weighted_rgpcd"]], 
    how="left", 
    on=["hydrologic_region","reporting_month"]
)

In [198]:
melt = pd.melt(
    merge_df, 
    id_vars=["supplier_name","hydrologic_region","reporting_month"], 
    value_vars=["calculated_r_gpcd","pop_weighted_rgpcd"]
)

In [199]:
base = alt.Chart(
    merge_df[(merge_df.supplier_name == "Los Angeles Department of Water and Power") & (merge_df.reporting_month > "2021-04-01")]
).encode(
    x=alt.X("yearmonth(reporting_month):O"),
)

bar = base.mark_bar(color="#83c6e0").encode(
    y=alt.Y("calculated_r_gpcd", stack=None)
)

avg_line = base.mark_line(interpolate='step', color='#1281aa').encode(
    y=alt.Y("pop_weighted_rgpcd")
)

goal_line = alt.Chart(pd.DataFrame({'y': [80]})).mark_rule(color="#b75a36",strokeDash=[10,11]).encode(y='y')

(bar + avg_line + goal_line).properties(title="LADWP residential water usage compared to regional average", width=600)

alt.LayerChart(...)

In [200]:
merge_df#[(merge_df.supplier_name == "Los Angeles Department of Water and Power") & (merge_df.reporting_month > "2021-04-01")]

,supplier_name,reporting_month,county,hydrologic_region,total_population_served,dwr_state_standard_level_corresponding_to_stage,calculated_r_gpcd,pop_weighted_rgpcd
0,East Bay Municipal Utilities District,2022-03-15,"Alameda,Contra Costa",San Francisco Bay,1430000.0,5.0,58.6280,59
1,East Bay Municipal Utilities District,2022-02-15,"Alameda,Contra Costa",San Francisco Bay,1430000.0,NaN,52.5704,55
2,East Bay Municipal Utilities District,2022-01-15,"Alameda,Contra Costa",San Francisco Bay,1420000.0,NaN,50.1604,48
3,East Bay Municipal Utilities District,2021-12-15,"Alameda,Contra Costa",San Francisco Bay,1420000.0,NaN,51.1677,48
4,East Bay Municipal Utilities District,2021-11-15,"Alameda,Contra Costa",San Francisco Bay,1420000.0,NaN,54.0265,50
...,...,...,...,...,...,...,...,...
37103,Thermalito Water and Sewer District,2021-05-15,Butte,Sacramento River,10312.0,NaN,228.7530,159
37104,Thermalito Water and Sewer District,2021-04-15,Butte,Sacramento River,10312.0,NaN,148.1860,123
37105,Thermalito Water and Sewer District,2021-03-15,Butte,Sacramento River,10312.0,NaN,89.5637,63
37106,Thermalito Water and Sewer District,2021-02-15,Butte,Sacramento River,10312.0,NaN,86.9818,72


In [201]:
sort_df = merge_df.sort_values(["reporting_month","supplier_name"])

In [202]:
sort_df.to_csv("../data/processed/district-level-residential-use.csv", index=False)